# Libraries

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import module.influx_utils as mod_inf
import pandas as pd
import fs
import os

# Change Time

In [23]:
# Example usage
ticks = 133539005217964000
print(mod_inf.ticks_to_rfc3339(ticks))  # Output: 2020-12-18T18:16:11Z

2024-03-03T00:48:41.796401Z


# Directories

In [24]:
MAIN_DIR = fs.open_fs("../data/processed/final")
DIR_TLG_SF_CSV = MAIN_DIR.getsyspath("TLG_SF.csv")
DIR_TLG_SF_CSV_INF = MAIN_DIR.getsyspath("TLG_SF_INF.csv")
DIR_TLG_SF_CSV_INF_temp = MAIN_DIR.getsyspath("TLG_SF_INF_temp.csv")

In [25]:
dF_TLG_SF = pd.read_csv(DIR_TLG_SF_CSV)
dF_TLG_SF.head()

pk_TimeStamp         Value                   tag_name
0  133543067468902405  12210.074219  Totalizador_Transferencia
1  133543079289260621   8033.640137  Totalizador_Transferencia
2  133543111229257914   4964.895996  Totalizador_Transferencia
3  133543134489783109   5118.520508  Totalizador_Transferencia
4  133543161780470776   5799.258301  Totalizador_Transferencia

In [26]:
dF_TLG_SF['_time'] = dF_TLG_SF['pk_TimeStamp'].apply(lambda x: mod_inf.ticks_to_rfc3339(x))

In [27]:
dF_TLG_SF.head()

pk_TimeStamp         Value                   tag_name  \
0  133543067468902405  12210.074219  Totalizador_Transferencia   
1  133543079289260621   8033.640137  Totalizador_Transferencia   
2  133543111229257914   4964.895996  Totalizador_Transferencia   
3  133543134489783109   5118.520508  Totalizador_Transferencia   
4  133543161780470776   5799.258301  Totalizador_Transferencia   

                         _time  
0  2024-03-07T17:39:06.890241Z  
1  2024-03-07T17:58:48.926062Z  
2  2024-03-07T18:52:02.925791Z  
3  2024-03-07T19:30:48.978313Z  
4  2024-03-07T20:16:18.047077Z

In [28]:
dF_TLG_SF_INF = dF_TLG_SF[['_time', 'Value', 'tag_name']].copy()
dF_TLG_SF_INF.columns = ['_time', '_value', '_field']
dF_TLG_SF_INF.to_csv(DIR_TLG_SF_CSV_INF, encoding='utf-8', index=False)
dF_TLG_SF_INF

_time        _value                     _field
0         2024-03-07T17:39:06.890241Z  12210.074219  Totalizador_Transferencia
1         2024-03-07T17:58:48.926062Z   8033.640137  Totalizador_Transferencia
2         2024-03-07T18:52:02.925791Z   4964.895996  Totalizador_Transferencia
3         2024-03-07T19:30:48.978313Z   5118.520508  Totalizador_Transferencia
4         2024-03-07T20:16:18.047077Z   5799.258301  Totalizador_Transferencia
...                               ...           ...                        ...
15285283  2024-06-26T19:19:03.130356Z     60.000000       PresionTransferencia
15285284  2024-06-26T19:43:01.562757Z      0.000000       PresionTransferencia
15285285  2024-06-26T19:43:01.614952Z      0.000000       PresionTransferencia
15285286  2024-06-26T19:52:48.150430Z     60.000000       PresionTransferencia
15285287  2024-06-27T15:16:00.656180Z     84.000000       PresionTransferencia

[15285288 rows x 3 columns]

In [29]:
dF_TLG_SF_INF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15285288 entries, 0 to 15285287
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   _time   object 
 1   _value  float64
 2   _field  object 
dtypes: float64(1), object(2)
memory usage: 349.9+ MB


In [30]:
measurement : str  = "old"
headers: str = f"#constant measurement,{measurement}\n{mod_inf.generate_headers_from_dataframe(dF_TLG_SF_INF)}"
file_path = DIR_TLG_SF_CSV_INF
temp_file_path = DIR_TLG_SF_CSV_INF_temp

# Read the existing content of the file
with open(file_path, 'r') as original_file:
    existing_content = original_file.read()

# Write the headers and then the existing content to a temporary file
with open(temp_file_path, 'w') as temp_file:
    temp_file.write(headers + '\n')
    temp_file.write(existing_content)

# Replace the original file with the temporary file
os.replace(temp_file_path, file_path)

PermissionError: [WinError 5] Access is denied: 'c:\\Users\\Lenovo\\Dropbox\\Luis Noguera\\Sucroliq\\Querétaro\\DataBase\\sqlite_tia_portal_etl\\data\\processed\\final\\TLG_SF_INF_temp.csv' -> 'c:\\Users\\Lenovo\\Dropbox\\Luis Noguera\\Sucroliq\\Querétaro\\DataBase\\sqlite_tia_portal_etl\\data\\processed\\final\\TLG_SF_INF.csv'